In [122]:
import numpy as np
import random

# **1) Model**

In [123]:
class DecisionTreeClassifier:
  def __init__(self, criterion='gini', max_depth=None, max_features=None):
    self.criterion = criterion
    self.max_depth = max_depth
    self.max_features = max_features

  def fit(self, X, y, sample_weights=None):
    X = np.array(X)
    y = np.array(y)
    assert len(X) > 0, "Length of X should be at least one"
    assert len(X) == len(y), "Lengths of X and y are not the same"
    assert len(X.shape) == 2, "X should be a bi-dimensional array"
    assert len(y.shape) == 1, "y should be a one-dimensional array"
    nb_samples, nb_features = X.shape
    self.n_features_in = nb_features

    num_classes = len(np.unique(y))
    assert max(y) + 1 == num_classes, "Error with class indexes"
    if sample_weights == None:
      sample_weights = np.ones(nb_samples)

    num_max_features = self.get_num_max_features(self.max_features)

    self.root = Node(n_features_in=self.n_features_in, num_max_features=num_max_features, num_classes=num_classes, criterion=self.criterion, depth=0, max_depth=self.max_depth)
    self.root.fit(X, y, sample_weights=sample_weights)

  def get_num_max_features(self, max_features):
    if max_features == None:
      return self.n_features_in

    elif isinstance(max_features, int):
      assert max_features > 0, "max_features should be greater than 0"
      assert max_features <= self.n_features_in, "max_features should be lower or equal to the total number of features"
      return max_features

    elif isinstance(max_features, float):
      assert max_features <= 1 and max_features > 0
      return max(1, int(max_features * self.n_features_in))

    elif isinstance(max_features, str):
      ratio_num_features = 0
      if max_features == 'sqrt' or max_features == 'auto':
        ratio_num_features = np.sqrt(self.n_features_in)

      elif max_features == 'log2':
        ratio_num_features = np.log2(self.n_features_in)

      else:
        raise Exception("max_features can be sqrt, log2 or auto")
      
      return self.get_num_max_features(ratio_num_features)

    
  def _predict_one(self, x):
    return self.root._predict_one(x)

  def predict(self, X):
    X = np.array(X)
    l = []
    for i, x in enumerate(X):
      pred = self._predict_one(x)
      l.append(pred)
    return np.array(l)



class Node:
  def __init__(self, n_features_in, num_max_features, num_classes, criterion='gini', depth=0, max_depth=None):
    self.n_features_in = n_features_in
    self.num_max_features = num_max_features
    self.num_classes = num_classes
    self.criterion = criterion
    self.criterion_function = self.get_criterion_function()
    self.depth = depth
    self.max_depth = max_depth

    self.is_leaf = False
    self.threshold = None
    self.majority_class = None
    self.chosen_feature_idx = None
    self.threshold = None
    self.left_node = None
    self.right_node = None

  def get_criterion_function(self):
    """Get the criterion function used (gini, entropy etc)"""
    if self.criterion == 'gini':
      return gini
    
    elif self.criterion == 'entropy':
      return entropy

    else:
      raise Exception("The criterion can only be gini or entropy")

  def config_leaf(self, y, sample_weights):
    """If the node must be a leaf, then it this functions computes the class predicted by this leaf"""
    self.is_leaf = True
    proportions = np.zeros(self.num_classes)
    for label, weight in zip(y, sample_weights):
      proportions[label] += weight

    self.majority_class = np.argmax(proportions)


  def compute_criterion_for_one_feature_for_all_thresholds(self, idx_feature, counter, sum_weights):
    """Given one feature, this functions computes all the criterion values (for all possible thresholds) and returns them as a list"""
    records = []
    thresholds = np.sort(list(counter.keys()))
    quantities_left = np.zeros(self.num_classes)
    quantities_right = np.zeros(self.num_classes)
    for thresh in thresholds:
      quantities_right += counter[thresh]['classes_quantities']

    for thresh in thresholds[:-1]:
      quantities_left += counter[thresh]['classes_quantities']
      quantities_right -= counter[thresh]['classes_quantities']
      total_quantities_left = sum(quantities_left)
      total_quantities_right = sum(quantities_right)
      criterion_value_left = self.criterion_function(quantities_left / total_quantities_left)
      criterion_value_right = self.criterion_function(quantities_right / total_quantities_right)

      assert total_quantities_left + total_quantities_right == sum_weights, "Quantities are not right"
      criterion_value = (total_quantities_left*criterion_value_left + total_quantities_right*criterion_value_right) / sum_weights

      records.append((idx_feature, thresh, criterion_value))

    return records


  def get_classes_quantities_per_value(self, X, y, sample_weights, idx_feature):
    """Given one feature, this function computes the classes distribution for every value of the feature"""
    feature_values = X[:, idx_feature]
    counter = {}
    for value, weight, label in zip(feature_values, sample_weights, y):
      try:
        counter[value]['classes_quantities'][label] += weight

      except:
        counter[value] = {}
        counter[value]['classes_quantities'] = np.zeros(self.num_classes)
        counter[value]['classes_quantities'][label] = weight

    return counter

  def create_new_trees(self, X, y, sample_weights, best_idx_feature, best_threshold):
    """Given one feature and on threshold, this function splits the data into two different parts to create two nodes, and it trains them recursively"""
    idx_right = np.argwhere(X[:, best_idx_feature] > best_threshold).flatten()
    idx_left = np.argwhere(X[:, best_idx_feature] <= best_threshold).flatten()

    assert min(len(idx_right), len(idx_left)) > 0, "Some part of the split data is empty"
    self.right_node = Node(n_features_in=self.n_features_in, 
                            num_max_features=self.num_max_features, 
                            num_classes=self.num_classes, 
                            criterion=self.criterion, 
                            depth=self.depth+1, 
                            max_depth=self.max_depth)
    self.left_node = Node(n_features_in=self.n_features_in, 
                            num_max_features=self.num_max_features, 
                            num_classes=self.num_classes, 
                            criterion=self.criterion, 
                            depth=self.depth+1, 
                            max_depth=self.max_depth)
    
    self.chosen_feature_idx = best_idx_feature
    self.threshold = best_threshold

    self.right_node.fit(X[idx_right], y[idx_right], sample_weights[idx_right])
    self.left_node.fit(X[idx_left], y[idx_left], sample_weights[idx_left])


  def fit(self, X, y, sample_weights):
    """Given some X and y data, this function trains a Node. It means that it looks for the best feature and threshold to split the data"""
    assert len(X) > 0, "Length of X should be at least one"
    assert len(X) == len(y), "Lengths of X and y are not the same"
    assert len(X.shape) == 2, "X should be a bi-dimensional array"
    assert len(y.shape) == 1, "y should be a one-dimensional array"

    if (self.depth == self.max_depth) or (len(np.unique(y)) == 1):
      self.config_leaf(y, sample_weights)

    else:
      criterion_values = []
      random_idx_features = random.sample(range(self.n_features_in), self.num_max_features)
      sum_weights = np.sum(sample_weights)
      for idx_feature in random_idx_features:
        counter = self.get_classes_quantities_per_value(X, y, sample_weights, idx_feature)
        criterion_values_for_one_feature = self.compute_criterion_for_one_feature_for_all_thresholds(idx_feature, counter, sum_weights)
        criterion_values.extend(criterion_values_for_one_feature)

      if len(criterion_values) >= 1:
        best_idx_feature, best_threshold, best_criterion_value = min(criterion_values, key=lambda x: x[2])
        self.create_new_trees(X, y, sample_weights, best_idx_feature, best_threshold)

      else:
        self.config_leaf(y, sample_weights)

  def _predict_one(self, x):
    if self.is_leaf:
      return self.majority_class
    
    else:
      node = None
      if (x[self.chosen_feature_idx] > self.threshold):
        node = self.right_node
      else:
        node = self.left_node

      return node._predict_one(x)
      

def gini(proportions):
  res = 0
  for p in proportions:
    res += p*(1-p)
  return res

def entropy(proportions):
  res = 0
  for p in proportions:
    if p > 0:
      res += (-p*np.log(p))
  return res

# **2) Test**

In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
import pandas as pd
import time
df = pd.read_csv('./drive/MyDrive/IA/Datasets/Star Classification/data.csv')
print('Shape : ', df.shape)
df.head(1)

Shape :  (100000, 18)


,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.2753,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171


In [126]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

encoder = {'GALAXY': 0, 'QSO': 1, 'STAR': 2}
df['class'] = df['class'].apply(lambda s: encoder[s])

X = df.drop(columns=['class'])
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [128]:
import sklearn.tree

print("--- Scikit-learn model ---")
t = time.time()
model_sklearn = sklearn.tree.DecisionTreeClassifier(max_depth=4)
model_sklearn.fit(X_train, y_train)
print(f"Time for training : {round(time.time() - t, 3)} sec")

preds = model_sklearn.predict(X_test)
score = accuracy_score(preds, y_test)
print(f"Accuracy : {score}")

--- Scikit-learn model ---
Time for training : 2.498 sec
Accuracy : 0.96555


In [129]:
print("--- My model ---")
t = time.time()
model = DecisionTreeClassifier(max_depth=4)
model.fit(X_train, y_train)
print(f"Time for training : {round(time.time() - t, 3)} sec")

preds = model.predict(X_test)
score = accuracy_score(preds, y_test)
print(f"Accuracy : {score}")

--- My model ---
Time for training : 96.1 sec
Accuracy : 0.96585
